# Import libraries and load data

In [9]:
!pip install tensorflow
!pip install keras

In [10]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [11]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

In [12]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset('./data/train')
valid_files, valid_targets = load_dataset('./data/valid')
test_files, test_targets = load_dataset('./data/test')

In [13]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

# Pre-process data

In [14]:
from PIL import ImageFile      

ImageFile.LOAD_TRUNCATED_IMAGES = True

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████████████████████████████████████████████| 836/836 [00:04<00:00, 191.99it/s]


In [15]:
print(len(train_files))

6680


# Modelling



A convolutional neural network (CNN) is a specific type of artificial neural network that uses perceptrons, a machine learning unit algorithm, for supervised learning, to analyze data. CNNs apply to image processing, natural language processing and other kinds of cognitive tasks. That is why I will be using it in this project for identification of the breed through the images of dogs.

In [16]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [17]:
model = Sequential() #the model uses one after another
model.add(Conv2D(filters=16, kernel_size=2, padding='same',activation='relu',input_shape=(224,224,3))) #Conv2D is convolution kernel that is wind with layers input which helps produce a tensor of outputs.
model.add(MaxPooling2D())
model.add(Conv2D(filters=32, kernel_size=2, padding='same',activation='relu'))
model.add(MaxPooling2D()) # MaxPooling2D is a pooling operation that calculates the maximum, or largest, value in each patch of each feature map
model.add(Conv2D(filters=64, kernel_size=2, padding='same',activation='relu'))
model.add(MaxPooling2D())
model.add(GlobalAveragePooling2D())
model.add(Dense(133,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)                0

In [18]:
#Compile and train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from keras.callbacks import ModelCheckpoint

epochs = 5 #one pass over the entire dataset
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Epoch 1/5
334/334 [==============================] - 82s 212ms/step - loss: 4.8868 - accuracy: 0.0068 - val_loss: 4.8686 - val_accuracy: 0.0108

Epoch 00001: val_loss improved from inf to 4.86864, saving model to saved_models\weights.best.from_scratch.hdf5
Epoch 2/5
334/334 [==============================] - 65s 196ms/step - loss: 4.8604 - accuracy: 0.0121 - val_loss: 4.8471 - val_accuracy: 0.0204

Epoch 00002: val_loss improved from 4.86864 to 4.84707, saving model to saved_models\weights.best.from_scratch.hdf5
Epoch 3/5
334/334 [==============================] - 67s 200ms/step - loss: 4.8236 - accuracy: 0.0189 - val_loss: 4.8203 - val_accuracy: 0.0180

Epoch 00003: val_loss improved from 4.84707 to 4.82026, saving model to saved_models\weights.best.from_scratch.hdf5
Epoch 4/5
334/334 [==============================] - 68s 202ms/step - loss: 4.7788 - accuracy: 0.0201 - val_loss: 4.7773 - val_accuracy: 0.0251

Epoch 00004: val_loss improved from 4.82026 to 4.77734, saving model to save

# Model using transfer learning

Using bottleneck features to convert the input X (image of size 224 x 224 x 3, for example) into the output Y with size 512 x 7 x 7. 

In [20]:
import requests

url = 'https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz'
r = requests.get(url)


In [21]:
with open('DogResnet50Data.npz', 'wb') as f:
    f.write(r.content)
    
bottleneck_features = np.load('DogResnet50Data.npz')
train_Resnet50 = bottleneck_features['train']
valid_Resnet50 = bottleneck_features['valid']
test_Resnet50 = bottleneck_features['test']

In [22]:
#Adding dropout to prevent the model from overfitting
Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=train_Resnet50.shape[1:]))
Resnet50_model.add(Dropout(0.3))
Resnet50_model.add(Dense(1024,activation='relu'))
Resnet50_model.add(Dropout(0.4))
Resnet50_model.add(Dense(133, activation='softmax'))
Resnet50_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               136325    
Total params: 2,234,501
Trainable params: 2,234,501
Non-trainable params: 0
_________________________________________________________________


 test accuracy of 80.8612% 

# Conclusion

I used CNN and then applied transfer learning with the intention to improve the model’s accuracy from 3.2% to 81% because of the small number of train images.